# Boston Housing Classification SVM with sequential forward selection (sfs)

In [ ]:
import sys
import pandas as pd
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.feature_selection import SelectFromModel, SequentialFeatureSelector
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
sys.path.append("..")

In [ ]:
inputFile = "../data/Boston_Housing_Data.csv"

## Read the data into DataFrame

In [ ]:
df = pd.read_csv(inputFile,delimiter=";")
print(df.info())

## Feature selection 

In [ ]:
df_features = df.drop(["MEDV","CAT"],axis=1) # drop label attribute from the features
df_labels = df[["CAT"]].copy()
display(df_features)
display(df_labels)

## Train test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_features,df_labels,test_size=0.3,random_state=1234)
display (X_train)
display (X_test) 
display (y_train)
display (y_test)

## Sequential Feature Selection

In [ ]:
sfs = SequentialFeatureSelector(LinearRegression(), n_features_to_select=5, cv=5)
sfs.set_output(transform="pandas")
sfs.fit(X_train,y_train)
sfs.transform(X_train)

## Pipelining

Support Vector Machine Classifier

In [ ]:
lsvc = LinearSVC(dual='auto') 

Build the pipeline

In [ ]:
feature_selector =SelectFromModel(estimator=LogisticRegression())
scaler = StandardScaler()
pipeline = make_pipeline(scaler, feature_selector, lsvc)
print (pipeline)

Build the paramGrid

In [ ]:
param_grid = dict(linearsvc__max_iter=[3000,5000,10000],\
                linearsvc__C=[1.0, 0.5, 0.1],\
                linearsvc__loss=["hinge"])                 \
                                 

Build the CrossValidator 

In [ ]:
grid_search = GridSearchCV(pipeline, param_grid=param_grid,cv=5, scoring="accuracy",return_train_score=True)
grid_search.fit(X_train, y_train["CAT"])

## Show best Model 

In [ ]:
print (grid_search.best_params_)
print (grid_search.best_estimator_)
print (grid_search.best_score_)

## Show selected features

In [ ]:
sup_columns = grid_search.best_estimator_["selectfrommodel"].get_support(indices=True)
df_features.columns[sup_columns]

## Test the model

In [ ]:
y_pred = grid_search.predict(X_test)
accuracy = accuracy_score(y_test,y_pred)
print("Test Error = " ,(1.0 - accuracy))

## Verify the best model

In [ ]:
pipeline.set_params(**grid_search.best_params_)
pipeline.fit(X_train,y_train["CAT"])
y_pred_opt = pipeline.predict(X_test)
accuracy = accuracy_score(y_test,y_pred_opt)
print("Test Error = " ,(1.0 - accuracy))

## Show selected features

In [ ]:
sup_columns = pipeline["selectfrommodel"].get_support(indices=True)
df_features.columns[sup_columns]